In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np

In [2]:
class SensorNodes:
    def __init__(self, x, y, initial_energy) -> None:
        self.x = x
        self.y = y
        self.id = None
        self.energy = initial_energy
        self.alive = True if self.energy > 0 else False
        self.cluster = None
        self.cluster_id = None
        self.cluster_head = None
        self.is_cluster_head = False
        self.distance_bs = float("inf")
    
class Cluster:
    def __init__(self, id, cluster_members = [], cluster_head = None) -> None:
        self.id = id
        self.cluster_members = cluster_members
        self.cluster_head = cluster_head
        

In [3]:
count = 0
class KNN_WOA:
    def __init__(self, num_nodes, base_station, lb, ub) -> None:
        self.num_nodes = num_nodes
        self.high_level_num = int(0.1 * num_nodes)
        self.mid_level_num = int(0.2 * num_nodes)
        self.low_level_num = int(0.7 * num_nodes)
        self.high_level_nodes = [SensorNodes(np.random.uniform(lb, ub), np.random.uniform(lb, ub), 2) for _ in range(self.high_level_num)]
        self.mid_level_nodes = [SensorNodes(np.random.uniform(lb, ub), np.random.uniform(lb, ub), 1.5) for _ in range(self.mid_level_num)]
        self.low_level_nodes = [SensorNodes(np.random.uniform(lb, ub), np.random.uniform(lb, ub), 1) for _ in range(self.low_level_num)]
        self.nodes = self.high_level_nodes + self.mid_level_nodes + self.low_level_nodes
        self.base_station = np.array(base_station)
        self.cluster_heads = []
        self.alive_nodes = [1 for _ in range(self.num_nodes)]
        self.lb = 0
        self.ub = 100
        
        self.iter = 0

        self.first_node_dead = None

        self.m = 5000
        self.E_b = 50 * (10 ** -9)
        self.E_fs = 10 * (10 ** -12) 
        self.E_mp = 13 * (10 ** -13)
        self.E_A = 5 * (10 ** -9)
        self.d_T = np.sqrt(self.E_fs / self.E_mp)

        for i in range(self.num_nodes):
            self.nodes[i].id = i
            if self.nodes[i].energy <= 0:
                self.alive_nodes[i] = 0
        for node in self.nodes:
            node.distance_bs = np.linalg.norm(np.array([node.x, node.y]) - self.base_station)

        # print("KNN_WOA Initialization Done!!!")
            
    def plotNodes(self):
        # print("Plotting Nodes Started!")
        x = [node.x for node in self.nodes]
        y = [node.y for node in self.nodes]
        colors = []
        for i in range(self.num_nodes):
            if self.nodes[i].energy > 1.5:
                colors.append('green')
            elif self.nodes[i].energy > 1:
                colors.append('blue')
            elif self.nodes[i].energy > 0:
                colors.append('red')
            else:
                colors.append('gray')
        # print("Plotting Nodes Done!")
        # for i in range(self.num_clusters):
        #     colors += [f'C{i}'] * (len([node for node in self.nodes if node.cluster == i]))

        # plt.figure(figsize=(8, 6))
        plt.title(f'Cluster Visualization {self.iter}')
        plt.xlabel('X Coordinate')
        plt.ylabel('Y Coordinate')
        plt.xlim(self.lb, self.ub)
        plt.ylim(self.lb, self.ub)
        # texts = [plt.text(x[i], y[i], f"{self.nodes[i].energy:.2f}", ha='right', va='bottom') for i in range(self.num_nodes)]
        plt.scatter(x, y, color=colors, marker='.')
        plt.scatter(self.base_station[0], self.base_station[1], color='black', marker='o')
        plt.grid(True)
    
    def plotClusterHeads(self, t):
        # print("Plotting Cluster Head Started!")
        x = [node.x for node in self.cluster_heads if node.alive]
        y = [node.y for node in self.cluster_heads if node.alive]

        # texts = [plt.text(x[i], y[i], f"CH {i}") for i in range(self.num_clusters)]
        # adjust_text(texts, x, y, arrowprops=dict(arrowstyle='->', color='gray', lw=0.5), autoalign='xy', expand_points=(1.2, 1.2))
        
        plt.scatter(x, y, c='black', marker=',')
        # plt.show()
        plt.savefig(f"knn_woa_t_{t}.png")
        plt.close()
        # print("Plotting Cluster Head Done!")

    def updateEnergy(self, node):
        d = np.sqrt((node.x - node.cluster_head.x) **2 + (node.y - node.cluster_head.y) ** 2)
        E_TX_SD_SN = self.m * self.E_b + self.m * self.E_fs * d**2 if d <= self.d_T else self.m * self.E_b + self.m * self.E_mp * d**4
        E_TX_CN = self.m * (self.E_A + self.E_b) + self.m * self.E_fs * d**2 if d <= self.d_T else self.m * (self.E_A + self.E_b) + self.m * self.E_mp * d**4
        # energy_fitness = (E_TX_CN / E_TX_SD_SN) + self.E_A
        depleted_energy = E_TX_CN if node.cluster_head else E_TX_SD_SN
        # depleted_energy = 0.2 if node.cluster_head else 0.1
        if node.energy <= 0:
            node.alive = 0
            if self.first_node_dead is None:
                self.first_node_dead = self.iter
            
        else:
            if node.cluster_head:
                node.energy -= depleted_energy
            else:
                node.energy -= depleted_energy
            node.energy = max(0, node.energy)
        for i in range(self.num_nodes):
            if not self.nodes[i].alive:
                self.alive_nodes[i] = 0

    def clusterFormation(self):
        scaler = StandardScaler()
        features = np.array([[node.x, node.y, node.distance_bs, node.energy] for node in self.nodes])
        features = scaler.fit_transform(features)

        min_clusters = max(1, int(0.05 * self.num_nodes))
        max_clusters = int(np.sqrt(self.num_nodes))
        self.num_clusters = np.random.randint(min_clusters, max_clusters)
        kmeans = KMeans(n_clusters = self.num_clusters, random_state=42).fit(features)

        cluster_assignments = kmeans.labels_
        self.num_clusters = len(set(cluster_assignments))
        self.clusters = [Cluster(i) for i in range(self.num_clusters)]
        
        for i in range(self.num_nodes):
            self.nodes[i].cluster = cluster_assignments[i]
            self.clusters[cluster_assignments[i]].cluster_members.append(self.nodes[i])

        for cluster in self.clusters:
            cluster.cluster_head = self.selectClusterHeadUsingWOA(cluster.cluster_members)
            if cluster.cluster_head:
                for node in cluster.cluster_members:
                    node.cluster_head = cluster.cluster_head
                    node.cluster_id = cluster.id
                self.cluster_heads.append(cluster.cluster_head)
                cluster.cluster_head.is_cluster_head = True
                # print(f"Cluster {cluster.id} Head: {cluster.cluster_head.id}")

        # for cluster in self.clusters:
        #     self.cluster_heads.append(self.nodes[np.random.choice([i for i in range(self.num_nodes) if cluster_assignments[i] == cluster.id])])
        #     cluster.cluster_head = self.cluster_heads[-1]
        
        for node in self.nodes:
            self.updateEnergy(node)

    def evaluate_fitness(self, virtual_point_position, nodes):
    # Define an objective function to evaluate the fitness of the virtual point
    # Evaluate the distance between the virtual point and each cluster member
        cluster_member_positions = np.array([[node.x, node.y] for node in nodes])
        energies = np.array([node.energy for node in nodes])
        distances = np.linalg.norm(cluster_member_positions - virtual_point_position, axis=1)
        # return distances.min()  # Fitness is the minimum distance to any cluster member
        # print("Distance of virtual point from all points: ", distances)
        fitness = np.sum(distances / (energies + 0.001))
        # print("Fitness: ", fitness)
        return fitness

    def selectClusterHeadUsingWOA(self, nodes):
        self.max_iterations = 10
        best_fitness = float('inf')
        whales_position = [[node.x, node.y] for node in nodes]
        best_solution = None
        best_fitness = float('inf')
        all_whales_positions = []  # Store positions of all whales for visualization

        for iteration in range(self.max_iterations):
            all_whales_positions.append(whales_position.copy())  # Store current positions for visualization
            a = 2 - 2 * iteration / self.max_iterations
            for i in range(len(nodes)):
                leader_index = np.random.randint(len(nodes))
                leader_position = whales_position[leader_index]

                A = 2 * a * np.random.random() - a
                C = 2 * np.random.random()
                p = np.random.random()
                leader_position = np.array(leader_position)
                whales_position = np.array(whales_position)
                # print(f"leaderposition: {type(leader_position)}")
                if p < 0.5:
                    new_position = leader_position - A * np.abs(C * leader_position - whales_position[i])
                else:
                    random_whale_index = np.random.randint(len(nodes))
                    leader_position = np.array(leader_position)
                    whales_position = np.array(whales_position)
                    D = np.abs(leader_position - whales_position[random_whale_index])
                    new_position = D * np.exp(0.1 * A) * np.cos(2 * np.pi * A) + leader_position

                new_position = np.clip(new_position, self.lb, self.ub)
                fitness = self.evaluate_fitness(new_position, nodes)

                if fitness < best_fitness:
                    best_solution = nodes[i]
                    best_fitness = fitness

                # if fitness < objective_function(whales_position[i], real_positions[i]):
                whales_position[i] = new_position
            # print("Best Solution: ", best_solution)
        # return best_solution, best_fitness, all_whales_positions
        # print("Best Solution: ", best_solution)
        return best_solution

    def run(self):
        t = 0
        while sum(self.alive_nodes) > 0:
            print(f"Time: {t}")
            self.iter = t
            self.clusterFormation()
            self.plotNodes()
            self.plotClusterHeads(t)
            t += 1
        print(f"First Node Dead: {self.first_node_dead}")
        print(f"Last Node Dead: {t}")

In [4]:
if __name__ == "__main__":
    num_nodes = 50
    base_station = [50, 50]
    lb = 0
    ub = 100
    knn_woa = KNN_WOA(num_nodes, base_station, lb, ub)
    knn_woa.run()

Time: 0
Time: 1


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Time: 2
Time: 3


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Time: 4


...

Time: 59


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Time: 60


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


First Node Dead: 4
Last Node Dead: 61
